## Классификация текстов: спам-фильтр для SMS

В этом задании вам предстоит взять открытый датасет с SMS-сообщениями, размеченными на спам ("spam") и не спам ("ham"), построить на нем классификатор текстов на эти два класса, оценить его качество с помощью кросс-валидации, протестировать его работу на отдельных примерах, и посмотреть, что будет происходить с качеством, если менять параметры вашей модели.

**1.** Загрузите датасет. Описание датасета можно посмотреть [здесь.](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection)  
**2.** Считайте датасет в Python (можете сразу грузить все в память, выборка небольшая), выясните, что используется в качестве разделителей и как проставляются метки классов.

In [2]:
import sys
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

In [5]:
data = pd.read_csv("data\SMSSpamCollection.txt", "\t", header=0, names=["label", "message"])
data.head()

,label,message
0,ham,Ok lar... Joking wif u oni...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,U dun say so early hor... U c already then say...
3,ham,"Nah I don't think he goes to usf, he lives aro..."
4,spam,FreeMsg Hey there darling it's been 3 week's n...


**3.** Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".

In [6]:
labels = data["label"] == "spam"
texts = data["message"]

**4.** Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получите из списка текстов матрицу признаков X.

In [9]:
vectorizer = CountVectorizer().fit(texts)
X = vectorizer.transform(texts).toarray()
y = labels

**5.** Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.

In [10]:
lr = LogisticRegression(random_state=2)
lr_score = cross_val_score(lr, X, y, cv=10, scoring="f1")

In [11]:
def save_answer_num(fname, number):
    """Функция сохранения в файл ответа, состоящего из одного числа"""
    with open(fname, "w") as fout:
        fout.write(str(number))

In [13]:
lr_mean_score = np.round(lr_score.mean(), 1)
save_answer_num("answers\w3_task_1\\1_mean_score.txt", lr_mean_score)
lr_mean_score

0.9

**6.** А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

"FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"

"FreeMsg: Txt: claim your reward of 3 hours talk time"

"Have you visited the last lecture on physics?"

"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"

"Only 99$"

Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.

In [17]:
test_messages = [
    "FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
    "FreeMsg: Txt: claim your reward of 3 hours talk time",
    "Have you visited the last lecture on physics?",
    "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
    "Only 99$"]

In [26]:
lr = LogisticRegression(random_state=2).fit(X, y)
test_x = vectorizer.transform(test_messages).toarray()
lr_predicts = lr.predict(test_x)
lr_predicts

array([ True,  True, False, False, False])

In [29]:
def save_answerArray(fname,array):
    with open(fname,"w") as fout:
        fout.write(" ".join([str(el) for el in array]))

In [30]:
save_answerArray("answers\w3_task_1\\2_lr_predicrions.txt", [int(x) for x in lr_predicts])

**7.** Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [33]:
ngram_ranges = [(2,2), (3,3), (1,3)]
def calculate_ngram_result(ngram_range, texts, vectorizer, model):
    vectorizer = vectorizer(ngram_range=ngram_range).fit(texts)
    X = vectorizer.transform(texts)
    return np.round(cross_val_score(model, X, y, cv=10, scoring="f1").mean(), 2)

ngram_results = [
    calculate_ngram_result(
        ngram_range,
        texts,
        CountVectorizer,
        LogisticRegression(random_state=2))
    for ngram_range in ngram_ranges]
ngram_results  

[0.82, 0.72, 0.92]

In [34]:
save_answerArray("answers\w3_task_1\\3_ngrams_scores.txt", ngram_results)

**8.** Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

По какой-то причине  обучение наивного байесовского классификатора через Pipeline происходит с ошибкой. Чтобы получить правильный ответ, отдельно  посчитайте частоты слов и обучите классификатор.  

In [35]:
ngram_nb_results = [
    calculate_ngram_result(
        ngram_range,
        texts,
        CountVectorizer,
        MultinomialNB())
    for ngram_range in ngram_ranges]
ngram_nb_results

[0.65, 0.38, 0.89]

In [36]:
save_answerArray("answers\w3_task_1\\4_ngrams_nb_scores.txt", ngram_nb_results)

**9.** Попробуйте использовать в логистической регрессии в качестве признаков Tf*idf из TfidfVectorizer на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к tf*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [38]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,1)).fit(texts)
X_tfidf = tfidf_vectorizer.transform(texts).toarray()
lr_tfidf = LogisticRegression(random_state=2)
lr_score_tfidf = cross_val_score(
    lr_tfidf,
    X_tfidf,
    y,
    cv=10,
    scoring="f1").mean()

In [39]:
print(f"Count vectorizer score: {lr_mean_score}")
print(f"Tfidf vectorizer score: {lr_score_tfidf}")

Count vectorizer score: 0.9
Tfidf vectorizer score: 0.8519368665928481


In [40]:
save_answer_num("answers\w3_task_1\\5_tfidf_score.txt", -1)